# Simply Parse HiResPerf Log and Read the Raw Counters.

Namely, this script doesn't process the data to calculate the rate of counters.

Right now, this is mainly used for `instructed_profile`, where we only log twice during each function running, one at the beginning and one at the ending.

In [40]:
import numpy as np
import pandas as pd

In [41]:
cpu_store_imc = 0

In [42]:
bin_file_path = "/home/ybyan/proj/rcs/bench/imc_reading/data/uncore_with_imc_30s_run5.bin"

In [43]:
def read_logs_to_numpy(file_path: str, use_imc: bool = False, use_offcore: bool = False) -> np.ndarray:
    c1_name = 'offcore_read' if use_offcore else 'llc_misses'
    c2_name = 'offcore_write' if use_offcore else 'sw_prefetch'

    if use_imc:
        # Matches "iQQQQQQQQ" -> int32, uint64, uint64, ...
        dt = np.dtype([
            ('cpu_id', np.int32),
            ('timestamp', np.uint64),
            ('stall_mem', np.uint64),
            ('inst_retire', np.uint64),
            ('cpu_unhalt', np.uint64),
            (f'{c1_name}', np.uint64),
            (f'{c2_name}', np.uint64),
            ('imc_read', np.uint64),
            ('imc_write', np.uint64),
        ])
    else:
        # Matches "iQQQQQQ"
        dt = np.dtype([
            ('cpu_id', np.int32),
            ('timestamp', np.uint64),
            ('stall_mem', np.uint64),
            ('inst_retire', np.uint64),
            ('cpu_unhalt', np.uint64),
            (f'{c1_name}', np.uint64),
            (f'{c2_name}', np.uint64),
        ])

    print("Reading binary file into NumPy array...")
    try:
        data = np.fromfile(file_path, dtype=dt)
        print(f"Read {len(data)} records to successfully.")
        return data
    except Exception as e:
        print(f"Error reading file with NumPy: {e}")
        return np.array([])


In [44]:
data_array = read_logs_to_numpy(bin_file_path, use_imc=True, use_offcore=False)

Reading binary file into NumPy array...
Read 112 records to successfully.


In [45]:
df = pd.DataFrame.from_records(data_array)
df.head()

,cpu_id,timestamp,stall_mem,inst_retire,cpu_unhalt,llc_misses,sw_prefetch,imc_read,imc_write
0,0,139497542929365,115231646966,686033992368,358428292935,684244350,1002177448,2122264,1944384
1,1,139497542929365,111524042503,1056066232751,449180683690,1185065123,986227461,0,0
2,2,139497542929365,56138830842,432932014674,201855024344,760460256,528793831,0,0
3,3,139497542929365,76587891829,890809841878,352810972844,990711175,1138617287,0,0
4,4,139497542929365,69147664413,140999686532188,284341571326,875762584,777263730,0,0


In [46]:
def calc_data_in_range(time_range: tuple[int, int], df: pd.DataFrame, use_offcore: bool) -> pd.DataFrame:
    c1_name = 'offcore_read' if use_offcore else 'llc_misses'
    c2_name = 'offcore_write' if use_offcore else 'sw_prefetch'
    c1_diff_name = f'{c1_name}_diff'
    c2_diff_name = f'{c2_name}_diff'

    start, end = time_range
    mask = (df['timestamp'] >= start) & (df['timestamp'] <= end)
    df_in_range = df.loc[mask]
    assert len(df_in_range["timestamp"].unique()) == 2, "There should be exactly two unique timestamps in the specified range."
    timestamp_min = df_in_range['timestamp'].min()
    timestamp_max = df_in_range['timestamp'].max()

    data_at_start = df.loc[df['timestamp'] == timestamp_min, ['cpu_id', f'{c1_name}', f'{c2_name}']]
    data_at_end = df.loc[df['timestamp'] == timestamp_max, ['cpu_id', f'{c1_name}', f'{c2_name}']]


    merged = pd.merge(data_at_end, data_at_start, on='cpu_id', suffixes=('_end', '_start'))
    
    print(f"{len(merged)} CPUs valid record found in the specified time range.")

    diff = pd.DataFrame({
        'cpu_id': merged['cpu_id'],
        f'{c1_diff_name}': merged[f'{c1_name}_end'] - merged[f'{c1_name}_start'],
        f'{c2_diff_name}': merged[f'{c2_name}_end'] - merged[f'{c2_name}_start']
    })
    print(f"Duration: {(timestamp_max - timestamp_min)/1999/1e6:.5f} s")
    
    diff_sum = diff[[c1_diff_name, c2_diff_name]].sum()
    print(diff_sum.to_dict())
    
    total_ = diff_sum.sum()
    print(f"Total diff: {total_}")
    print(f"Data transferred ((c1+c2) * 64): {total_ * 64 / 1e6:.2f} MB") 
    
    imc_col_names = ['imc_read', 'imc_write']

    start_imc = df.loc[(df['timestamp'] == timestamp_min) & (df['cpu_id'] == cpu_store_imc), imc_col_names]
    end_imc = df.loc[(df['timestamp'] == timestamp_max) & (df['cpu_id'] == cpu_store_imc), imc_col_names]

    if not start_imc.empty and not end_imc.empty:
        imc_read_diff = int(end_imc['imc_read'].values[0]) - int(start_imc['imc_read'].values[0])
        imc_write_diff = int(end_imc['imc_write'].values[0]) - int(start_imc['imc_write'].values[0])
        print(f"IMC read diff: {imc_read_diff}, IMC write diff: {imc_write_diff}")
        print(f"IMC total transferred ((read+write) * 64): {(imc_read_diff + imc_write_diff) * 64 / 1e6:.2f} MB")
    else:
        print("IMC data not available for the specified cpu_id and timestamps.")

    return df_in_range

In [47]:
df["timestamp"].min(), df["timestamp"].max()

(np.uint64(139497542929365), np.uint64(139557557491155))

In [48]:
out_df = calc_data_in_range((df["timestamp"].min(), df["timestamp"].max()), df, use_offcore=False)

56 CPUs valid record found in the specified time range.
Duration: 30.02229 s
{'llc_misses_diff': 5144710, 'sw_prefetch_diff': 371152}
Total diff: 5515862
Data transferred ((c1+c2) * 64): 353.02 MB
IMC read diff: 23024461, IMC write diff: 19502550
IMC total transferred ((read+write) * 64): 2721.73 MB


In [129]:
out_df[out_df["timestamp"] == out_df["timestamp"].min()][["offcore_read", "offcore_write"]].sum()

offcore_read     26775110868
offcore_write    22289366204
dtype: uint64

In [130]:
out_df[out_df["timestamp"] == out_df["timestamp"].max()][["offcore_read", "offcore_write"]].sum()

offcore_read     26776070893
offcore_write    22289853626
dtype: uint64

In [169]:
start_df_off_write = out_df[out_df["timestamp"] == out_df["timestamp"].min()]["offcore_write"]
start_df_off_write

0     816383773
1     769016776
2     444463357
3     859453066
4     624802118
5     527995735
6     432125802
7     477124832
8     510222545
9     436345263
10    292991543
11    354392082
12    327528118
13    449891015
14    446925436
15    310085781
16    268138555
17    345656854
18    169495365
19    179085975
20    232774378
21    395028640
22    223941518
23    404095225
24    205827393
25    257459372
26    619097187
27    656692573
28    349810880
29    358656255
30    284304447
31    457855532
32    386685977
33    584186901
34    496100503
35    644487610
36    786498424
37    515737534
38    577782001
39    447011842
40    472068766
41    463912961
42    674060110
43    742482009
44    534142546
45    430437873
46    465956180
47    456265588
48     31619119
49     79249276
50     18185345
51     10124948
52     16948714
53     15469572
54      8478478
55     11990566
Name: offcore_write, dtype: uint64

In [170]:
end_df_off_write = out_df[out_df["timestamp"] == out_df["timestamp"].max()]["offcore_write"]
end_df_off_write

56     816522124
57     769057275
58     444470291
59     859572286
60     624837717
61     527998893
62     432126210
63     477147504
64     510222560
65     436350387
66     292991553
67     354392972
68     327528895
69     449891051
70     446925945
71     310086170
72     268341716
73     345781226
74     169714490
75     179086451
76     232776186
77     395183522
78     224155004
79     407032070
80     205827432
81     257459434
82     619100138
83     656714562
84     349812216
85     358666506
86     284326997
87     457924375
88     386727605
89     584205791
90     496117375
91     644489073
92     786516930
93     515751082
94     577794575
95     447022295
96     472109930
97     463934941
98     674096700
99     744936592
100    534351675
101    430934442
102    466355629
103    456702977
104     34671943
105     79325050
106     18191246
107     10125696
108     16978502
109     15490514
110      8501818
111     11993927
Name: offcore_write, dtype: uint64

In [172]:
diff_off_write = end_df_off_write.values - start_df_off_write.values
diff_off_write

array([ 138351,   40499,    6934,  119220,   35599,    3158,     408,
         22672,      15,    5124,      10,     890,     777,      36,
           509,     389,  203161,  124372,  219125,     476,    1808,
        154882,  213486, 2936845,      39,      62,    2951,   21989,
          1336,   10251,   22550,   68843,   41628,   18890,   16872,
          1463,   18506,   13548,   12574,   10453,   41164,   21980,
         36590, 2454583,  209129,  496569,  399449,  437389, 3052824,
         75774,    5901,     748,   29788,   20942,   23340,    3361],
      dtype=uint64)

In [65]:
(21381145782 - 21379910376 + (17452380814 - 17451201521)) * 64 / 1e6

154.540736

In [66]:
out_df[out_df["timestamp"] == out_df["timestamp"].min()]

,cpu_id,timestamp,stall_mem,inst_retire,cpu_unhalt,offcore_read,offcore_write,imc_read,imc_write
0,0,70166658242131,78582169181,549616358975,262556625457,589668695,802003642,8286291,4213673
1,1,70166658242131,71999261183,550230685290,251168416227,974927160,762919737,0,0
2,2,70166658242131,44630191212,336403943611,156203331009,659566975,441051014,0,0
3,3,70166658242131,49119201056,543873199558,216040061055,726045529,855698442,0,0
4,4,70166658242131,45485640460,140748461760297,178635085512,694351901,618626671,0,0
5,5,70166658242131,37624043832,140927685663922,144368123626,532225781,527407657,0,0
6,6,70166658242131,32126188385,275082406259,112301667784,485267833,426726369,0,0
7,7,70166658242131,40757034605,353772524073,145936788026,625644795,476224946,0,0
8,8,70166658242131,46996996723,140738095401583,59442168409,572211244,496561866,0,0
9,9,70166658242131,30839231451,300664349411,117477363308,440618315,427105459,0,0
